In [ ]:
from helper import *

import pandas as pd
import numpy as np
import os
import csv
import sys
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [8, 8]
# use LaTeX fonts in the plot
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})
%matplotlib inline

In [ ]:
EVAL_DIR = "/home/felix/todo/osm-tmp"
MLP_METHODS = ["kmeans", "merge", "hop"]
# MLP_LEVELS = [[int(2 ** i)] for i in np.arange(8.0, 13.5, 0.5)]
MLP_LEVELS = [[int(2 ** i)] for i in np.arange(8.0, 13.5, 1.0)]
FAST_QUERY_METHODS = ["crp", "pch", "prp"]
QUERY_METHODS = ["normal", "bi"] + FAST_QUERY_METHODS
print(MLP_METHODS, "with", MLP_LEVELS)

In [ ]:
with open(EVAL_DIR + "/log.json") as file:
    tmp = file.readlines()
    log = pd.read_json(" ".join(tmp), orient='index')

In [ ]:
# create command string to not match against list
log['command_string'] = log['command'].agg(lambda x: ' '.join(map(str, x)))

In [ ]:
fig, ax = plt.subplots()

for method in MLP_METHODS:
    x = list()
    y = list()
    for level in MLP_LEVELS:
        lines = log[(log['command_string'].str.contains("prp_pre")) & (log['command_string'].str.contains(method + "-" + "_".join(map(str, level))))]
        if lines.shape[0] == 0:
            continue
        # take last line of execution
        line = lines.iloc[-1]
        runtime = line['time'].replace(" seconds", "")
        x.append(level)
        y.append(float(runtime))
    ax.plot(x, y, color=plot_get(method))
    ax.scatter(x, y, color=plot_get(method), label=method)
    
ax.set_xlabel("number of partitions")
ax.set_ylabel("runtime in seconds")
ax.legend(loc='upper right')
fig.savefig("pre-runtime.pgf", bbox_inches="tight")